In [ ]:
import  numpy as np
import  pandas as pd

In [3]:
'''
高频率的数据聚合到低频率被称为向下采样,向下采样中,目标频率必须是原频率的子区间
低频率的数据聚合到高频率被称为向上采样,向上采样中,目标频率必须是原频率的父区间
又不是所有的重新采样都属于向上或向下,W-WEN(weekly on Wednesday,每周三)转换到W-FRI(每周五)既不是向上采样也不是向下采样)
'''

'\n高频率的数据聚合到低频率被称为向下采样,向下采样中,目标频率必须是原频率的子区间\n低频率的数据聚合到高频率被称为向上采样,向上采样中,目标频率必须是原频率的父区间\n又不是所有的重新采样都属于向上或向下,W-WEN(weekly on Wednesday,每周三)转换到W-FRI(每周五)既不是向上采样也不是向下采样)\n'

In [4]:
rng = pd.date_range('2000-01-01', periods=100, freq='d')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.head()

2000-01-01    0.311131
2000-01-02    0.370231
2000-01-03    0.638511
2000-01-04   -1.043456
2000-01-05   -0.815414
Freq: D, dtype: float64

In [5]:
# 直接对时间序列进行操作
s = ts.resample(rule='BM').asfreq() # 类似于groupby分组
'''
rule : str
        The offset string or object representing target conversion.
'''
print(s)
print(type(s.index))

2000-01-31    1.500307
2000-02-29   -0.511212
2000-03-31    0.858972
2000-04-28         NaN
Freq: BM, dtype: float64
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [6]:
s1 = ts.resample('M', kind='period').mean() # rule不能等于'BM'
'''
kind : {'timestamp', 'period'}, optional, default None
    Pass 'timestamp' to convert the resulting index to a
    `DateTimeIndex` or 'period' to convert it to a `PeriodIndex`.
    By default the input representation is retained.
'''
print(s1)
print(s1.index)
print(type(s1.index))

2000-01    0.287473
2000-02   -0.106542
2000-03   -0.005554
2000-04   -0.552478
Freq: M, dtype: float64
PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04'], dtype='period[M]', freq='M')
<class 'pandas.core.indexes.period.PeriodIndex'>


In [52]:
def top(df):
	return  df.sum()

rng1 = pd.date_range('2000', periods=366, freq='d')
frame = pd.DataFrame(np.reshape(s, (len(rng1), 2)), 
                     index=rng1, columns=list('AB'))
print(frame.head())

                   A         B
2000-01-01  0.656553 -0.519493
2000-01-02 -0.185357  0.156206
2000-01-03 -1.689500  0.094476
2000-01-04  0.050731 -0.069762
2000-01-05  0.100679 -0.488057


In [53]:
frame.resample('M').sum().head() # 聚合

,A,B
2000-01-31,1.338741,-5.131865
2000-02-29,-2.054102,0.078789
2000-03-31,-1.003090,-2.379426
2000-04-30,1.849164,-0.432397
2000-05-31,-5.066477,1.971307


In [54]:
frame.resample('BM').sum().head() # 这里rule可以等于'BM'

,A,B
2000-01-31,1.338741,-5.131865
2000-02-29,-2.054102,0.078789
2000-03-31,-1.003090,-2.379426
2000-04-28,2.056667,-1.194756
2000-05-31,-5.273981,2.733666


In [55]:
frame.resample('BM').apply(top).head() # 类似于groupby函数映射apply

,A,B
2000-01-31,1.338741,-5.131865
2000-02-29,-2.054102,0.078789
2000-03-31,-1.003090,-2.379426
2000-04-28,2.056667,-1.194756
2000-05-31,-5.273981,2.733666


In [56]:
# 如果传输的是函数或函数名的列表,将得到一个列名是这些函数名的DataFrame
frame.resample('M').agg([np.sum, np.mean]).head() # 类似于groupby中agg函数进行数据聚合

A                   B          
                 sum      mean       sum      mean
2000-01-31  1.338741  0.043185 -5.131865 -0.165544
2000-02-29 -2.054102 -0.070831  0.078789  0.002717
2000-03-31 -1.003090 -0.032358 -2.379426 -0.076756
2000-04-30  1.849164  0.061639 -0.432397 -0.014413
2000-05-31 -5.066477 -0.163435  1.971307  0.063591

In [63]:
frame.resample('M').agg({'A': ['sum', 'std'],
                         'B': ['mean', lambda x: np.std(x, ddof=1)]}).head() # 不同的函数应用到不同的列中

A                   B           
                 sum       std      mean <lambda_0>
2000-01-31  1.338741  1.007099 -0.165544   1.016248
2000-02-29 -2.054102  0.863285  0.002717   0.996236
2000-03-31 -1.003090  1.140368 -0.076756   1.030643
2000-04-30  1.849164  1.102423 -0.014413   0.836517
2000-05-31 -5.066477  0.967861  0.063591   1.027284

In [64]:
frame.resample('M').agg({'A': np.sum, 'B': 'std'}) # DatetimeIndexResampler自带的聚合函数要加上''

,A,B
2000-01-31,1.338741,1.016248
2000-02-29,-2.054102,0.996236
2000-03-31,-1.003090,1.030643
2000-04-30,1.849164,0.836517
2000-05-31,-5.066477,1.027284
2000-06-30,-7.041840,1.013769
2000-07-31,-2.349089,1.175865
2000-08-31,-1.060093,0.900933
2000-09-30,8.111693,1.059731
2000-10-31,7.214518,0.956085


In [65]:
frame.resample('M').agg([('foo', 'mean'), ('bar', 'std')]) # 每个元祖的一个元素作为DataFrame的列名
                                                        # 二元元组的列表是一种有序的对应关系

A                   B          
                 foo       bar       foo       bar
2000-01-31  0.043185  1.007099 -0.165544  1.016248
2000-02-29 -0.070831  0.863285  0.002717  0.996236
2000-03-31 -0.032358  1.140368 -0.076756  1.030643
2000-04-30  0.061639  1.102423 -0.014413  0.836517
2000-05-31 -0.163435  0.967861  0.063591  1.027284
2000-06-30 -0.234728  1.101193 -0.287603  1.013769
2000-07-31 -0.075777  0.996948  0.110524  1.175865
2000-08-31 -0.034197  0.955518  0.131633  0.900933
2000-09-30  0.270390  1.084543 -0.050951  1.059731
2000-10-31  0.232726  0.871797 -0.037001  0.956085
2000-11-30 -0.001738  0.986776  0.392470  1.069059
2000-12-31 -0.064183  1.009852 -0.109528  0.892736

In [66]:
# 具有多个lambdas的Groupby聚合
frame.resample('M').agg([
     ('隐函数一', lambda x: x.iloc[0] - x.iloc[1]),
    ('隐函数二', lambda x: x.iloc[0] + x.iloc[1])
])

A                   B          
                隐函数一      隐函数二      隐函数一      隐函数二
2000-01-31  0.841910  0.471195 -0.675699 -0.363287
2000-02-29  0.679387 -0.213486 -0.678465  0.156600
2000-03-31  3.498618 -1.130736  0.891332 -1.669961
2000-04-30  0.045984  0.431801 -1.873845 -0.304114
2000-05-31  1.948153  2.520154  0.925431  3.318312
2000-06-30 -0.026573 -2.401980  1.632258 -1.107615
2000-07-31  1.158613  1.766176 -0.751371 -2.251912
2000-08-31  1.367499 -0.164350  0.956639 -1.694538
2000-09-30  0.276254 -0.966950 -1.025361  1.205313
2000-10-31 -1.734489  1.425553 -1.587432 -1.704826
2000-11-30 -2.215389 -2.544291 -1.089977  1.479312
2000-12-31  3.302726 -0.544128  1.529340  2.283659